In [4]:
#####################
####PREPROCESSING####
#####################

import pandas as pd 
import string
import numpy as np

np.random.seed(41)

def ngrams(doc, n): 
    doc = doc.lower().translate(str.maketrans('', '', ".!()-\""))
    doc = doc.split()
    grams = []
    for i in range(len(doc) - n + 1): 
        gram = " ".join(doc[i:i+n])
        grams.append(gram)
    return grams

ngrams("hello darkness my old friend i've come to talk with you again .", 3)

['hello darkness my',
 'darkness my old',
 'my old friend',
 "old friend i've",
 "friend i've come",
 "i've come to",
 'come to talk',
 'to talk with',
 'talk with you',
 'with you again']

In [5]:
# Get training and test data

data = pd.read_csv('songdata.csv')
kanyetext = data.loc[data['artist'] == 'Kanye West', 'text']
draketext = data.loc[data['artist'] == 'Drake', 'text']
eminemtext = data.loc[data['artist'] == 'Eminem', 'text']
migostext = data.loc[data['artist'] == 'Migos', 'text']
lilwaynetext = data.loc[data['artist'] == 'Lil Wayne', 'text']

text = [kanyetext, draketext, eminemtext, migostext, lilwaynetext]
X = []
y = []
for i in range(len(text)):
    for j in text[i]:
        gram = ngrams(j, 3)
        for k in range(len(gram)):
            X.append(gram[k])
            y.append(i)

# append each word as a training data 
     #remove these characters from the text
X = np.array(X)
y = np.array(y)

print(len(y))
print(len(X))

205882
205882


In [6]:
# shuffle! 
randomize = np.arange(len(y))
np.random.shuffle(randomize)
X = X[randomize]
y = y[randomize]

# Train and test split 50/50 split
n_train, n_test = int(len(X) *0.7), int(len(X) * 0.3)
text_train = X[:n_train]
text_test = y[n_train : n_train+n_test]
y_train = y[:n_train]
y_test = y[n_train : n_train+n_test]

In [7]:
print(text_train[0])
print("\nClass: ", y_train[0])

x2 and i'm

Class:  1


In [8]:
####################
#### spacy time ####
####################

import spacy

nlp = spacy.load('en')

# SymbolTable & LSTM code adapted 
# from https://github.com/ix-ai-s1-17/lstm-examples/blob/master/AGNewsKeras.ipynb

class SymbolTable:
    """Wrapper for dict to encode unknown symbols"""

    def __init__(self, starting_symbol=2, unknown_symbol=1): 
        self.s       = starting_symbol
        self.unknown = unknown_symbol
        self.d       = dict()

    def lookup_add(self, w):
        if w not in self.d:
            self.d[w] = self.s
            self.s += 1
        return self.d[w]

    def lookup(self, w, strict=False):
        return self.d[w] if strict else self.d.get(w, self.unknown)

    def reverse(self):
        r = {v: k for k, v in self.d.iteritems()}
        r[0], r[1] = '~~NONE~~', '~~UNKNOWN~~'
        return r

    def num_words(self):
        return len(self.d)

    def num_symbols(self):
        return self.s

In [9]:
# Parse all text
text_train_parsed = [nlp(str(s)) for s in text_train]
print(len(text_train_parsed))
text_test_parsed = [nlp(str(s)) for s in text_test]
print(len(text_test_parsed))

144117
61764


In [10]:
# Convert text to integer symbols
symbol_table = SymbolTable()

def preprocess_text(parsed_text, symbol_table, init=True):
    mapper = symbol_table.lookup_add if init else symbol_table.lookup
    return [[mapper(w.text.strip().lower()) for s in t.sents for w in s] for t in parsed_text]

In [11]:
symbols_train = preprocess_text(text_train_parsed, symbol_table, True)
symbols_test = preprocess_text(text_test_parsed, symbol_table, False)

print(len(symbols_train))
print(len(symbols_test))

144117
61764


In [12]:
from keras.preprocessing import sequence

MAX_LENGTH = 50 #not sure what this does!
print(len(symbols_train))
print(len(symbols_train[0]))
x_train = sequence.pad_sequences(symbols_train, maxlen=MAX_LENGTH)
print(x_train.shape)
x_test = sequence.pad_sequences(symbols_test, maxlen=MAX_LENGTH)

144117
4
(144117, 50)


In [13]:
n_classes = len(set(y))
print(n_classes)

5


In [14]:
#########################
#### Building Model ####
########################

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM, Bidirectional
from keras.optimizers import Adam

d = 128

model = Sequential()
model.add(Embedding(symbol_table.num_symbols(), output_dim=d))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
#model.add(LSTM(128))
#model.add(Dropout(0.2))
#model.add(Embedding(output_dim=d, input_length=1))
#model.add(LSTM(128))
#model.add(Dropout(0.2))
#model.add(Embedding(output_dim=d, input_length=1))
#model.add(LSTM(128))
#model.add(Dropout(0.2))
#model.add(Embedding(symbol_table.num_symbols(), output_dim=d))
#model.add(LSTM(128))
#model.add(Dropout(0.2))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=20)


Train on 144117 samples, validate on 61764 samples
Epoch 1/20
128640/144117 [=========================>....] - ETA: 140s - loss: 1.2670 - acc: 0.4605

KeyboardInterrupt: 